In [1]:
import os
import numpy as np
import pandas as pd
import random
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
ratings.isna().sum()

user_id      0
movie_id     0
ratings      0
timestamp    0
dtype: int64

1) 데이터 준비와 전처리

In [3]:
using_cols = ['user_id', 'movie_id', 'ratings']
ratings = ratings[using_cols]
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies.isna().sum()

movie_id    0
title       0
genre       0
dtype: int64

In [8]:
movies['title']

0                         Toy Story (1995)
1                           Jumanji (1995)
2                  Grumpier Old Men (1995)
3                 Waiting to Exhale (1995)
4       Father of the Bride Part II (1995)
                       ...                
3878               Meet the Parents (2000)
3879            Requiem for a Dream (2000)
3880                      Tigerland (2000)
3881               Two Family House (2000)
3882                 Contender, The (2000)
Name: title, Length: 3883, dtype: object

2) 분석해 봅시다.
ratings에 있는 유니크한 영화 개수
ratings에 있는 유니크한 사용자 수
가장 인기 있는 영화 30개(인기순)

In [9]:
ratings['movie_id'].nunique()

3628

In [10]:
ratings['user_id'].nunique()

6039

In [11]:
movies_count = ratings.groupby('movie_id')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [12]:
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [13]:
user_median = ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [14]:
my_favorite = ['Princess Mononoke, The (Mononoke Hime) (1997)' ,'Tarzan (1999)' ,'Toy Story (1995)', 'Chicken Run (2000)',
               'Saludos Amigos (1943)', 'Iron Giant, The (1999)', 'Pokémon: The First Movie (1998)', "Bug's Life, A (1998)",
               'Sleeping Beauty (1959)']
title_to_idx = {v:k for k,v in enumerate(movies.title)}
mvidlist=[]
countlist=[]
for i in my_favorite:
    mvidlist.append(title_to_idx[i]+1)

for i in range(len(mvidlist)):
    countlist.append(random.randint(3,5))

new_user=ratings['user_id'].nunique()+2

my_playlist = pd.DataFrame({'user_id': [new_user]*len(mvidlist), 'movie_id': mvidlist, 'counts':countlist})

if not ratings.isin({'user_id':[new_user]})['user_id'].any():
    ratings = ratings.append(my_playlist)

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts
1000208,6040,1097,4
0,6041,2932,3
1,6041,2619,3
2,6041,1,4
3,6041,3683,4
4,6041,3543,5
5,6041,2693,3
6,6041,2986,3
7,6041,2287,5
8,6041,2028,3


4) CSR matrix를 직접 만들어 봅시다.

In [15]:
num_user = ratings['user_id'].nunique()
num_movie_id = ratings['movie_id'].nunique()

print(num_user)
print(num_movie_id)
print(ratings.user_id.shape)
print(ratings.movie_id.shape)
print(ratings.counts.shape)

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user+2, num_movie_id+325))
csr_data

6040
3628
(836487,)
(836487,)
(836487,)


<6042x3953 sparse matrix of type '<class 'numpy.longlong'>'
	with 836487 stored elements in Compressed Sparse Row format>

5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=333, regularization=0.01, use_gpu=False, iterations=33, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.longlong'>'
	with 836487 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/33 [00:00<?, ?it/s]

6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [19]:
zimin, toystory = new_user, title_to_idx['Toy Story (1995)']+1
zimin_vector, toystory_vector = als_model.user_factors[zimin], als_model.item_factors[toystory]

In [20]:
np.dot(zimin_vector, toystory_vector)

0.80356336

In [21]:
sleeping = title_to_idx['Sleeping Beauty (1959)']+1
sleeping_vector = als_model.item_factors[sleeping]
np.dot(zimin_vector, sleeping_vector)

0.84523314

In [22]:
toy2 = title_to_idx['Toy Story 2 (1999)']+1
toy2_vector = als_model.item_factors[toy2]
np.dot(zimin_vector, toy2_vector)

-0.026584737

In [23]:
madmax = title_to_idx["Mad Max (1979)"]+1
madmax_vector = als_model.item_factors[madmax]
np.dot(zimin_vector, madmax_vector)

0.0022707782

7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [24]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]+1
    similar_movie = als_model.similar_items(movie_id)
    print(similar_movie)
    similar_movie = [movies.title[i[0]-1] for i in similar_movie]
    return similar_movie

In [25]:
get_similar_movie('Toy Story (1995)')

[(1, 1.0000001), (3114, 0.35112786), (106, 0.2717791), (1473, 0.27108428), (3796, 0.26836526), (966, 0.26820958), (588, 0.26696247), (1324, 0.26277018), (3344, 0.26266742), (179, 0.26206094)]


['Toy Story (1995)',
 'Mr. Death: The Rise and Fall of Fred A. Leuchter Jr. (1999)',
 'Muppet Treasure Island (1996)',
 'Hollow Reed (1996)',
 'Original Kings of Comedy, The (2000)',
 'Moonlight Murder (1936)',
 'Tough and Deadly (1995)',
 'Cape Fear (1962)',
 'Bear, The (1988)',
 'Mallrats (1995)']

In [26]:
get_similar_movie('Toy Story 2 (1999)')

[(3046, 1.0000001), (461, 0.70432043), (2127, 0.6631904), (3631, 0.65958387), (1139, 0.65494555), (1509, 0.64824075), (1549, 0.647526), (2198, 0.6461587), (3282, 0.64140934), (439, 0.64022696)]


['Toy Story 2 (1999)',
 'Hard Target (1993)',
 'Dirty Work (1998)',
 'Starman (1984)',
 'T-Men (1947)',
 'Shiloh (1997)',
 'Cop Land (1997)',
 "Butcher's Wife, The (1991)",
 'Raisin in the Sun, A (1961)',
 'Demolition Man (1993)']

In [27]:
get_similar_movie('Princess Mononoke, The (Mononoke Hime) (1997)')

[(2932, 1.0), (2925, 0.6568234), (2131, 0.6417236), (3473, 0.6356566), (2923, 0.6223373), (3588, 0.6211431), (3738, 0.6177139), (681, 0.61245334), (2239, 0.6114278), (2285, 0.6091892)]


['Princess Mononoke, The (Mononoke Hime) (1997)',
 'Thunderball (1965)',
 'Phoenix (1998)',
 'Third World Cop (1999)',
 'Live and Let Die (1973)',
 'Lured (1947)',
 "MacKenna's Gold (1969)",
 'Operation Dumbo Drop (1995)',
 'One Tough Cop (1998)',
 'Enemy of the State (1998)']

In [28]:
get_similar_movie('Sixth Sense, The (1999)')

[(2694, 1.0000001), (2683, 0.41028017), (2541, 0.39730713), (216, 0.39384887), (2392, 0.38807368), (2555, 0.38209626), (1911, 0.38156945), (3004, 0.35943583), (2546, 0.35585806), (2335, 0.34769642)]


['Sixth Sense, The (1999)',
 'From the Hip (1987)',
 'King of Masks, The (Bian Lian) (1996)',
 'Babysitter, The (1995)',
 'Texas Chainsaw Massacre 2, The (1986)',
 "Trippin' (1999)",
 'Friday the 13th Part VI: Jason Lives (1986)',
 'Moonstruck (1987)',
 'Chopping Mall (a.k.a. Killbots) (1986)',
 'First Blood (1982)']

8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [29]:
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(new_user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(3114, 0.25617084),
 (223, 0.20706007),
 (1394, 0.19545297),
 (2396, 0.18764879),
 (968, 0.17638807),
 (1253, 0.17507079),
 (3095, 0.16562507),
 (2761, 0.1650364),
 (3342, 0.15838124),
 (1248, 0.15722421),
 (2859, 0.15606864),
 (2355, 0.15565313),
 (2699, 0.15341689),
 (162, 0.14801344),
 (1301, 0.13984857),
 (3608, 0.13850883),
 (1217, 0.13537808),
 (921, 0.13478026),
 (2424, 0.13414983),
 (1019, 0.13372645)]

In [30]:
for i in range(5):
    print(movies.title[movie_recommended[i][0]-1])

Mr. Death: The Rise and Fall of Fred A. Leuchter Jr. (1999)
Disclosure (1994)
Evita (1996)
Trick or Treat (1986)
Nothing Personal (1995)


In [31]:
explain = als_model.explain(new_user, csr_data, itemid=movie_recommended[0][0])
explain

(0.25320911308936855,
 [(1, 0.2010342720644717),
  (2986, 0.04569027611290182),
  (2693, 0.029580789222498377),
  (3683, 0.020875922812737913),
  (2619, 0.00023879770390393217),
  (2932, -0.000271537802345056),
  (2028, -0.00046197079263789013),
  (2287, -0.01860513113180528),
  (3543, -0.024872305100356942)],
 (array([[ 0.60813318,  0.11469229,  0.1388796 , ...,  0.17195604,
           0.14380857,  0.16184122],
         [ 0.06974819,  0.57503128,  0.09345346, ...,  0.107777  ,
           0.10378021,  0.10060385],
         [ 0.08445729,  0.06966708,  0.56696509, ...,  0.10318129,
           0.09997425,  0.0857504 ],
         ...,
         [ 0.10457218,  0.08169718,  0.09245351, ...,  0.50578597,
           0.00061264, -0.00636917],
         [ 0.08745476,  0.0761706 ,  0.08635261, ...,  0.10044544,
           0.49820242,  0.00501481],
         [ 0.09842101,  0.0764123 ,  0.0804957 , ...,  0.09642631,
           0.09356689,  0.506809  ]]),
  False))

하이퍼파라미터를 바꾸어 보았을 때 결과의 수치가 변하기는 하지만 결과 목록은 거의 변하지 않았다.
rating.counts 값 3~5 에 따른 차이는 거의 없다.
선호 영화 목록에 따라 결과가 확연히 변화하였다.

1. CSR matrix가 정상적으로 만들어졌다.

사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

 - CSR matrix가 정상적으로 만들어졌는데 정확한 사이즈 수치가 왜 저렇게 된건지 모르겠다.


2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.

사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

 - 수치들은 그럴듯하게 나타났지만 예상과 많이 다른 결과가 나타났다. 결과로 나온 영화 제목을 검색해 간략한 정보를 보았을 때 비슷한 영화가 아닌것 같고, 데이터셋에 있는 시리즈작품은 유사한 영화로 나오지 않았다. 그리고 선호하는 영화를 편향되게 선택하였지만 추천 결과는 선호 영화와 동떨어진 결과가 나왔다.